In [1]:
%pip install pyautogen llama-index llama-index-tools-wikipedia llama-index-readers-wikipedia wikipedia

Note: you may need to restart the kernel to use updated packages.


In [10]:
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.tools.wikipedia import WikipediaToolSpec
from autogen.agentchat.contrib.llamaindex_conversable_agent import LLamaIndexConversableAgent
import os
import autogen
import dotenv

In [3]:
dotenv.load_dotenv()

True

In [4]:
# config_list = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={"tags": ["gpt-3.5-turbo"]},  # comment out to get all
# )


# When using a single openai endpoint, you can use the following:
config_list = [{"model": "gpt-4o-mini", "api_key": os.getenv("OPENAI_API_KEY")}]

In [5]:
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [6]:
embed_model = OpenAIEmbedding(
    model="text-embedding-ada-002",
    temperature=0.0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [7]:
Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
# create a react agent to use wikipedia tool
wiki_spec = WikipediaToolSpec()
# Get the search wikipedia tool
wikipedia_tool = wiki_spec.to_tool_list()[1]

In [9]:
location_specialist = ReActAgent.from_tools(tools=[wikipedia_tool], llm=llm, max_iterations=10, verbose=True)

In [11]:
llm_config = {
    "temperature": 0,
    "config_list": config_list,
}

In [12]:
trip_assistant = LLamaIndexConversableAgent(
    "trip_specialist",
    llama_index_agent=location_specialist,
    system_message="You help customers finding more about places they would like to visit. You can use external resources to provide more details as you engage with the customer.",
    description="This agents helps customers discover locations to visit, things to do, and other details about a location. It can use external resources to provide more details. This agent helps in finding attractions, history and all that there is to know about a place",
)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="ALWAYS",
    code_execution_config=False,
)

In [14]:
groupchat = autogen.GroupChat(
    agents=[trip_assistant, user_proxy],
    messages=[],
    max_round=500,
    speaker_selection_method="round_robin",
    enable_clear_history=True,
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [17]:
chat_result = user_proxy.initiate_chat(
    manager,
    message="""
Plan a 4 day trip iternary of Jaipur, India.
""",
)

Admin (to chat_manager):


Plan a 4 day trip iternary of Jaipur, India.


--------------------------------------------------------------------------------

Next speaker: trip_specialist


>>>>>>>> USING AUTO REPLY...
> Running step 6e07ae04-c01e-4178-97c0-6c360deb63d3. Step input: 
Plan a 4 day trip iternary of Jaipur, India.

Thought: The current language of the user is: English. I can provide a 4-day itinerary for a trip to Jaipur, India without needing to use a tool.
Answer: Here’s a suggested 4-day itinerary for your trip to Jaipur, India:

### Day 1: Arrival and City Exploration
- **Morning**: Arrive in Jaipur. Check into your hotel and freshen up.
- **Afternoon**: Visit the City Palace, a beautiful blend of Mughal and Rajput architecture. Explore the museum and the courtyards.
- **Evening**: Head to Jantar Mantar, an astronomical observatory. Afterward, stroll through the local markets like Johari Bazaar and Bapu Bazaar for shopping and local snacks.

### Day 2: Forts and Palaces